## Generate centroids of multipolygons

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
import couchdb
import os
import json

In [2]:
# load boundaries of states and SUAs
state_gdf = gpd.read_file("../data/Digital boundaries/state/STE_2016_AUST.shp")
sua_gdf = gpd.read_file("../data/Digital boundaries/SUA/SUA_2016_AUST.shp")

In [5]:
state_gdf.shape

(9, 4)

In [4]:
sua_gdf.head()

,SUA_CODE16,SUA_NAME16,AREASQKM16,geometry
0,1000,Not in any Significant Urban Area (NSW),787692.8137,"MULTIPOLYGON (((159.06752 -31.50458, 159.06744..."
1,1001,Albury - Wodonga,628.2471,"POLYGON ((146.93218 -36.01951, 146.93245 -36.0..."
2,1002,Armidale,274.5995,"POLYGON ((151.58534 -30.50992, 151.58587 -30.5..."
3,1003,Ballina,73.9758,"MULTIPOLYGON (((153.57534 -28.85495, 153.57530..."
4,1004,Batemans Bay,94.3022,"MULTIPOLYGON (((150.26035 -35.74771, 150.26035..."


In [15]:
list(sua_gdf['SUA_NAME16'])


['Not in any Significant Urban Area (NSW)',
 'Albury - Wodonga',
 'Armidale',
 'Ballina',
 'Batemans Bay',
 'Bathurst',
 'Bowral - Mittagong',
 'Broken Hill',
 'Camden Haven',
 'Central Coast',
 'Coffs Harbour',
 'Dubbo',
 'Forster - Tuncurry',
 'Goulburn',
 'Grafton',
 'Griffith',
 'Kempsey',
 'Lismore',
 'Lithgow',
 'Morisset - Cooranbong',
 'Mudgee',
 'Muswellbrook',
 'Nelson Bay',
 'Newcastle - Maitland',
 'Nowra - Bomaderry',
 'Orange',
 'Parkes',
 'Port Macquarie',
 'Singleton',
 'St Georges Basin - Sanctuary Point',
 'Sydney',
 'Tamworth',
 'Taree',
 'Ulladulla',
 'Wagga Wagga',
 'Wollongong',
 'Not in any Significant Urban Area (Vic.)',
 'Bacchus Marsh',
 'Bairnsdale',
 'Ballarat',
 'Bendigo',
 'Colac',
 'Echuca - Moama',
 'Geelong',
 'Gisborne - Macedon',
 'Horsham',
 'Melbourne',
 'Melton',
 'Mildura - Wentworth',
 'Moe - Newborough',
 'Portland',
 'Sale',
 'Shepparton - Mooroopna',
 'Swan Hill',
 'Traralgon - Morwell',
 'Wangaratta',
 'Warragul - Drouin',
 'Warrnambool',
 'N

In [16]:
def centroid_state(gdf, name):
    gdf['geometry'][8] = state_gdf['geometry'][8].geoms[0]
    gdf['centroids'] = gdf.centroid
    gdf['lat'] = gdf['centroids'].y
    gdf['lng'] = gdf['centroids'].x
    gdf['name'] = gdf[name].str.lower()
    gdf = gdf[['name', 'lat', 'lng']]
    return gdf

In [17]:
def centroid_sua(gdf, name):
    gdf['centroids'] = gdf.centroid
    gdf['lat'] = gdf['centroids'].y
    gdf['lng'] = gdf['centroids'].x
    gdf['name'] = gdf[name]
    gdf = gdf[['name', 'lat', 'lng']]
    return gdf

In [18]:
state_gdf_finalized = centroid_state(state_gdf, 'STE_NAME16')
state_gdf_finalized['name'][8] = 'offshore territories'
sua_gdf_finalized = centroid_sua(sua_gdf, 'SUA_NAME16')

In [19]:
list(state_gdf_finalized['name'])

['new south wales',
 'victoria',
 'queensland',
 'south australia',
 'western australia',
 'tasmania',
 'northern territory',
 'australian capital territory',
 'offshore territories']

In [20]:
geo_center = pd.concat([state_gdf_finalized, sua_gdf_finalized])
geo_center.head(20)

,name,lat,lng
0,new south wales,-32.165600,147.008154
1,victoria,-36.850094,144.304288
2,queensland,-22.567052,144.545851
3,south australia,-30.108162,135.831738
4,western australia,-25.466471,122.181775
5,tasmania,-41.936457,146.595152
6,northern territory,-19.414739,133.364913
7,australian capital territory,-35.488766,149.002523
8,offshore territories,-12.176305,96.830483
0,Not in any Significant Urban Area (NSW),-32.150416,146.947104


In [21]:
geo_center.shape

(119, 3)

In [22]:
with open('../data//Digital boundaries/geo_center.json', 'w') as f:
    geo_center.to_json(f, orient='records')

In [23]:
user = 'admin'
psw = 'qazwsxedc'
node_ip = '172.26.130.9'
# connect to the CouchDB server
server = couchdb.Server(f'http://{user}:{psw}@{node_ip}:5984')

# select the database
db = server['geocenter']

i = 0
package = []

with open(r"../data//Digital boundaries/geo_center.json", "r") as f:
    data = json.load(f)
    for item in data:
        db.save(item)